## Imports

In [3]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [4]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [5]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [6]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

1     (10201 Collins Ave #1801s, Bal Harbour, FL 331...
2     (360 Ocean Dr #302s, Key Biscayne, FL 33149, U...
3     (10155 Collins Ave, Bal Harbour, FL 33154, USA...
4     (2901 S Bayshore Dr #13b, Miami, FL 33133, USA...
5     (718 Valencia Ave Unit 305, Coral Gables, FL 3...
6     (1395 Brickell Ave #3406, Miami, FL 33131, USA...
7     (200 Biscayne Blvd Way APT 1109, Miami, FL 331...
8     (15811 Collins Ave APT 2103, Sunny Isles Beach...
9     (6365 Collins Ave APT 3507, Miami Beach, FL 33...
10    (2020 N Bayshore Dr APT 801, Miami, FL 33137, ...
Name: loc, dtype: object

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [11]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Oceana Bal Harbour\r10201 Collins Ave 1801S\rB...,2/25/2022,7/11/2022,14,"$12,350,000.00",4185,"$2,951.02",Mark Zilbert,Mark Zilbert,Tracy Galya,Douglas Elliman,Oceana Bal Harbour 10201 Collins Ave 1801S Bal...,Oceana Bal Harbour,10201 Collins Ave 1801S Bal Harbour,"(10201 Collins Ave #1801s, Bal Harbour, FL 331...","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0
2,Oceana Key Biscayne\r360 Ocean Dr 302S\rKey Bi...,1/15/2022,7/15/2022,65,"$6,950,000.00",3358,"$2,069.68",Claudio Iglesias,"Brickell Realty Group, LLC",Cassio Galiza PA,"Podium Realty Group, Inc",Oceana Key Biscayne 360 Ocean Dr 302S Key Bisc...,Oceana Key Biscayne,360 Ocean Dr 302S Key Biscayne,"(360 Ocean Dr #302s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
3,Bal Harbour 101 Condo\r10155 Collins Ave 507\r...,4/21/2022,7/11/2022,26,"$2,450,000.00",2525,$970.30,Jeffrey London,Fortune International Realty,Jannette Zafrani,Elite International Realty,Bal Harbour 101 Condo 10155 Collins Ave 507 Ba...,Bal Harbour,101 Condo 10155 Collins Ave 507 Bal Harbour,"(10155 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8931472, -80.1231444, 0.0)",25.893147,-80.123144,0.0
4,Yacht Harbour Condo\r2901 S Bayshore Dr 13B\rC...,6/6/2022,7/11/2022,1,"$2,250,000.00",1587,"$1,417.77",Carole Smith,"Compass Florida, LLC",Maite Alvarez,"Compass Florida, LLC",Yacht Harbour Condo 2901 S Bayshore Dr 13B Coc...,Yacht Harbour Condo,2901 S Bayshore Dr 13B Coconut Grove,"(2901 S Bayshore Dr #13b, Miami, FL 33133, USA...","(25.7277525, -80.2394939, 0.0)",25.727753,-80.239494,0.0
5,Biltmore Parc Condo\r718 Valencia Ave 305\rCor...,3/1/2022,7/12/2022,83,"$2,199,000.00",2345,$937.74,Anniella Tabraue,Rosval Realty,Marilu Perez-Perez,Specialist Realty Group Inc,Biltmore Parc Condo 718 Valencia Ave 305 Coral...,Biltmore Parc Condo,718 Valencia Ave 305 Coral Gables,"(718 Valencia Ave Unit 305, Coral Gables, FL 3...","(25.7468655, -80.2698381, 0.0)",25.746865,-80.269838,0.0
6,Brickell Arch\r1395 Brickell Ave 3406\rMiami,3/9/2022,7/15/2022,64,"$2,075,000.00",2485,$835.01,Susana Rosado,Williams Island Realty LLC,Ruben Pena,Douglas Elliman,Brickell Arch 1395 Brickell Ave 3406 Miami,Brickell Arch,1395 Brickell Ave 3406 Miami,"(1395 Brickell Ave #3406, Miami, FL 33131, USA...","(25.7603404, -80.1920171, 0.0)",25.760340,-80.192017,0.0
7,Epic Residence\r200 Biscayne Blvd Way 1109\rMiami,3/7/2022,7/11/2022,50,"$1,700,000.00",1489,"$1,141.71",Carlos Uribe,"Inmobiliata, LLC",Carlos Uribe,"Inmobiliata, LLC",Epic Residence 200 Biscayne Blvd Way 1109 Miami,Epic Residence,200 Biscayne Blvd Way 1109 Miami,"(200 Biscayne Blvd Way APT 1109, Miami, FL 331...","(25.7705273, -80.1895438, 0.0)",25.770527,-80.189544,0.0
8,Trump Towers\r15811 Collins Ave 2103\rSunny Is...,12/21/2021,7/13/2022,82,"$1,550,000.00",1935,$801.03,Cassio Filho,FACCIN Group Realty,Cassio Filho,FACCIN Group Realty,Trump Towers 15811 Collins Ave 2103 Sunny Isle...,Trump Towers,15811 Collins Ave 2103 Sunny Isles Beach,"(15811 Collins Ave APT 2103, Sunny Isles Beach...","(25.9232597, -80.1218135, 0.0)",25.923260,-80.121814,0.0
9,Akoya\r6365 Collins Ave 3507\rMiami Beach,6/5/2022,7/11/2022,1,"$1,400,000.00",1322,"$1,059.00",Nils Jucaud,Charles Rutenberg Realty,Gustavo Duquesne,"Compass Florida, LLC",Akoya 6365 Collins Ave 3507 Miami Beach,Akoya,6365 Collins Ave 3507 Miami Beach,"(6365 Collins Ave APT 3507, Miami Beach, FL 33...","(25.8460388, -80.1196447, 0.0)",25.846039,-80.119645,0.0
10,P0aramount Bay Condo\r2020 N Bayshore Dr 801\r...,5/2/2022,7/12/2022,56,"$1,350,000.00",1588,$850.13,Yadira Henriquez Saba,BHHS EWM Realty,Marion Ott,Douglas Elliman,P0aramount Bay Condo 2020 N Bayshore Dr 801 Miami,P,0aramount Bay Condo 2020 N Bayshore Dr 801 Miami,"(2020 N Bayshore Dr APT 801, Miami, FL 33137, ...",

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [17]:
df.at[10,'building_name']=('Paramount Bay')
df.at[10,'address']=('2020 N Bayshore Dr 801')

In [19]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 10th - July 16th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [21]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [22]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Condo_Sales_week_ending_07_16_22
